<a href="https://colab.research.google.com/github/culiacanai/Aprende_Python_con_GoogleColab/blob/main/notebooks/13_Intro_a_Machine_Learning.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🤖 Intro a Machine Learning

### Aprende Python con Google Colab — por [Culiacan.AI](https://culiacan.ai)

**Nivel:** 🔴 Avanzado  
**Duración estimada:** 90 minutos  
**Requisitos:** Haber completado los Notebooks [11 NumPy](11_Intro_a_NumPy.ipynb) y [12 Análisis de Datos Real](12_Analisis_de_Datos_Real.ipynb)

---

En este notebook vas a:
- Entender qué es Machine Learning y los tipos que existen
- Conocer Scikit-learn, la librería #1 de ML en Python
- Entrenar tu primer modelo de regresión lineal
- Entrenar tu primer modelo de clasificación
- Evaluar modelos con métricas reales
- Entender el flujo completo: datos → entrenamiento → evaluación → predicción

> 💡 **Machine Learning** es darle a una computadora la capacidad de aprender patrones a partir de datos, sin programar reglas explícitas. Es la base de la Inteligencia Artificial moderna.


---

## 0. Preparación


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings("ignore")

# Scikit-learn (ya viene instalado en Colab)
import sklearn
print(f"✅ Scikit-learn versión: {sklearn.__version__}")

---

## 1. ¿Qué es Machine Learning?

### Programación tradicional vs Machine Learning

```
Programación tradicional:
  Datos + Reglas → Programa → Resultado

Machine Learning:
  Datos + Resultados → Algoritmo → Reglas (modelo)
```

### Tipos de Machine Learning

| Tipo | Qué hace | Ejemplo |
|------|----------|---------|
| **Supervisado** | Aprende de datos etiquetados | Predecir precio de casa según sus características |
| **No supervisado** | Encuentra patrones sin etiquetas | Agrupar clientes similares |
| **Refuerzo** | Aprende por prueba y error | Un robot que aprende a caminar |

En este notebook nos enfocaremos en **aprendizaje supervisado**, que tiene dos grandes categorías:

| Categoría | Variable objetivo | Ejemplo |
|-----------|------------------|---------|
| **Regresión** | Número continuo | Predecir ventas, precios, temperaturas |
| **Clasificación** | Categoría | Predecir si un cliente compra o no, tipo de producto |

### El flujo de Machine Learning

```
1. Obtener datos
2. Explorar y limpiar
3. Dividir en entrenamiento y prueba
4. Elegir un modelo
5. Entrenar el modelo
6. Evaluar el modelo
7. Hacer predicciones
```


---

## 2. Nuestros datos: prediciendo esperanza de vida

Vamos a usar el dataset Gapminder para predecir la esperanza de vida de un país basándonos en su PIB per cápita y población.


In [ ]:
# Cargar datos de Gapminder
import plotly.express as px
gapminder = px.data.gapminder()

# Usar datos de 2007 para nuestro primer modelo
df = gapminder[gapminder["year"] == 2007].copy()
df = df.rename(columns={
    "country": "pais", "continent": "continente", "year": "anio",
    "lifeExp": "esperanza_vida", "pop": "poblacion", "gdpPercap": "pib_per_capita",
})

print(f"📊 Dataset: {len(df)} países en 2007")
print(f"\nColumnas: {list(df.columns)}")
df.head()

In [ ]:
# Exploración rápida
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(df["esperanza_vida"], bins=20, color="#2E86AB", edgecolor="white")
axes[0].set_title("Distribución: Esperanza de Vida", fontweight="bold")
axes[0].set_xlabel("Años")

axes[1].hist(np.log10(df["pib_per_capita"]), bins=20, color="#A23B72", edgecolor="white")
axes[1].set_title("Distribución: log(PIB per cápita)", fontweight="bold")
axes[1].set_xlabel("log₁₀(USD)")

axes[2].scatter(df["pib_per_capita"], df["esperanza_vida"], alpha=0.6, color="#F18F01", edgecolors="white")
axes[2].set_title("PIB vs Esperanza de Vida", fontweight="bold")
axes[2].set_xlabel("PIB per cápita (USD)")
axes[2].set_ylabel("Esperanza de vida (años)")

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

---

## 3. Dividir datos: entrenamiento y prueba

**¿Por qué dividir?** Si evaluamos el modelo con los mismos datos que usamos para entrenarlo, no sabemos si realmente aprendió o si solo "memorizó". Es como estudiar con el examen ya resuelto.

```
Datos totales (100%)
├── Entrenamiento (80%) → El modelo aprende de estos
└── Prueba (20%) → Evaluamos con estos (el modelo nunca los vio)
```


In [ ]:
from sklearn.model_selection import train_test_split

# Preparar datos
# X = features (variables predictoras)
# y = target (variable a predecir)

# Usamos log del PIB porque la relación es más lineal así
df["log_pib"] = np.log10(df["pib_per_capita"])
df["log_pop"] = np.log10(df["poblacion"])

X = df[["log_pib"]].values  # Una sola feature por ahora
y = df["esperanza_vida"].values

# Dividir: 80% entrenamiento, 20% prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"📊 División de datos:")
print(f"  Total:        {len(X)} países")
print(f"  Entrenamiento: {len(X_train)} países ({len(X_train)/len(X)*100:.0f}%)")
print(f"  Prueba:        {len(X_test)} países ({len(X_test)/len(X)*100:.0f}%)")

---

## 4. Tu primer modelo: Regresión Lineal

La regresión lineal busca la mejor línea recta que explique la relación entre las variables.

$$y = mx + b$$

Donde:
- $y$ = esperanza de vida (lo que queremos predecir)
- $x$ = log(PIB per cápita) (lo que usamos para predecir)
- $m$ = pendiente (cuánto cambia $y$ por cada unidad de $x$)
- $b$ = intercepto (valor de $y$ cuando $x = 0$)


In [ ]:
from sklearn.linear_model import LinearRegression

# Crear y entrenar el modelo
modelo_lr = LinearRegression()
modelo_lr.fit(X_train, y_train)

# Parámetros aprendidos
pendiente = modelo_lr.coef_[0]
intercepto = modelo_lr.intercept_

print(f"📐 Modelo entrenado:")
print(f"  Pendiente (m): {pendiente:.2f}")
print(f"  Intercepto (b): {intercepto:.2f}")
print(f"\n  Fórmula: esperanza_vida = {pendiente:.2f} × log(PIB) + ({intercepto:.2f})")
print(f"\n  Interpretación: por cada 10x aumento en PIB per cápita,")
print(f"  la esperanza de vida aumenta ~{pendiente:.1f} años")

In [ ]:
# Hacer predicciones
y_pred_train = modelo_lr.predict(X_train)
y_pred_test = modelo_lr.predict(X_test)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Datos de entrenamiento
axes[0].scatter(X_train, y_train, alpha=0.6, color="#2E86AB", edgecolors="white", label="Datos reales")
axes[0].plot(np.sort(X_train, axis=0), modelo_lr.predict(np.sort(X_train, axis=0)),
             color="#C73E1D", linewidth=2.5, label="Modelo")
axes[0].set_title("Entrenamiento", fontweight="bold")
axes[0].set_xlabel("log₁₀(PIB per cápita)")
axes[0].set_ylabel("Esperanza de vida (años)")
axes[0].legend()

# Datos de prueba
axes[1].scatter(X_test, y_test, alpha=0.6, color="#F18F01", edgecolors="white", label="Datos reales")
axes[1].plot(np.sort(X_test, axis=0), modelo_lr.predict(np.sort(X_test, axis=0)),
             color="#C73E1D", linewidth=2.5, label="Modelo")
axes[1].set_title("Prueba (datos nuevos)", fontweight="bold")
axes[1].set_xlabel("log₁₀(PIB per cápita)")
axes[1].set_ylabel("Esperanza de vida (años)")
axes[1].legend()

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

plt.suptitle("Regresión Lineal: PIB vs Esperanza de Vida", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

---

## 5. Evaluar el modelo: ¿qué tan bueno es?

### Métricas de regresión

| Métrica | Qué mide | Ideal |
|---------|----------|-------|
| **R²** | % de variación explicada por el modelo | Cercano a 1 |
| **MAE** | Error promedio absoluto | Cercano a 0 |
| **RMSE** | Error promedio (penaliza errores grandes) | Cercano a 0 |


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Métricas en datos de PRUEBA (lo que importa)
r2 = r2_score(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"📊 Evaluación del modelo (datos de prueba):")
print(f"  R² Score:  {r2:.3f}  ({r2*100:.1f}% de la variación explicada)")
print(f"  MAE:       {mae:.2f} años  (error promedio)")
print(f"  RMSE:      {rmse:.2f} años  (error promedio penalizado)")

# Comparar train vs test
r2_train = r2_score(y_train, y_pred_train)
print(f"\n  R² Train:  {r2_train:.3f}")
print(f"  R² Test:   {r2:.3f}")
if r2_train - r2 > 0.1:
    print("  ⚠️ Posible overfitting (el modelo memoriza en vez de aprender)")
else:
    print("  ✅ El modelo generaliza bien")

In [ ]:
# Visualizar errores
errores = y_test - y_pred_test

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Predicción vs Real
axes[0].scatter(y_test, y_pred_test, alpha=0.6, color="#2E86AB", edgecolors="white")
lim = [min(y_test.min(), y_pred_test.min()) - 2, max(y_test.max(), y_pred_test.max()) + 2]
axes[0].plot(lim, lim, "--", color="#C73E1D", linewidth=2, label="Predicción perfecta")
axes[0].set_xlabel("Valor real (años)")
axes[0].set_ylabel("Predicción (años)")
axes[0].set_title("Predicción vs Realidad", fontweight="bold")
axes[0].legend()

# Distribución de errores
axes[1].hist(errores, bins=10, color="#A23B72", edgecolor="white", alpha=0.8)
axes[1].axvline(0, color="#C73E1D", linestyle="--", linewidth=2)
axes[1].set_xlabel("Error (años)")
axes[1].set_ylabel("Frecuencia")
axes[1].set_title("Distribución de Errores", fontweight="bold")

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

print(f"Error máximo: {np.max(np.abs(errores)):.1f} años")
print(f"Error promedio: {np.mean(np.abs(errores)):.1f} años")

---

## 6. Mejorar el modelo: más features

Un solo feature (PIB) no captura toda la historia. Agreguemos más variables.


In [ ]:
# Preparar datos con más features
# Usaremos datos de TODOS los años para tener más datos de entrenamiento
df_full = gapminder.copy()
df_full = df_full.rename(columns={
    "country": "pais", "continent": "continente", "year": "anio",
    "lifeExp": "esperanza_vida", "pop": "poblacion", "gdpPercap": "pib_per_capita",
})

# Crear features
df_full["log_pib"] = np.log10(df_full["pib_per_capita"])
df_full["log_pop"] = np.log10(df_full["poblacion"])

# One-hot encoding para continentes (convertir categorías a números)
continentes_dummies = pd.get_dummies(df_full["continente"], prefix="cont", drop_first=True)
df_full = pd.concat([df_full, continentes_dummies], axis=1)

# Features
feature_cols = ["log_pib", "log_pop", "anio"] + list(continentes_dummies.columns)
X_full = df_full[feature_cols].values
y_full = df_full["esperanza_vida"].values

print(f"Features utilizados ({len(feature_cols)}):")
for i, col in enumerate(feature_cols):
    print(f"  {i+1}. {col}")

print(f"\nDatos totales: {len(X_full)} registros")

# Dividir
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42
)

In [ ]:
# Entrenar modelo con más features
modelo_multi = LinearRegression()
modelo_multi.fit(X_train_f, y_train_f)

# Evaluar
y_pred_multi = modelo_multi.predict(X_test_f)
r2_multi = r2_score(y_test_f, y_pred_multi)
mae_multi = mean_absolute_error(y_test_f, y_pred_multi)
rmse_multi = np.sqrt(mean_squared_error(y_test_f, y_pred_multi))

print(f"📊 Modelo con múltiples features:")
print(f"  R²:   {r2_multi:.3f} (antes: {r2:.3f})")
print(f"  MAE:  {mae_multi:.2f} años (antes: {mae:.2f})")
print(f"  RMSE: {rmse_multi:.2f} años (antes: {rmse:.2f})")
print(f"\n  {'✅ Mejoró!' if r2_multi > r2 else '❌ No mejoró'}")

# Importancia de cada feature (coeficientes)
print(f"\n📐 Coeficientes del modelo:")
for col, coef in zip(feature_cols, modelo_multi.coef_):
    print(f"  {col:<25} {coef:>8.3f}")

---

## 7. Más allá de la regresión lineal

Scikit-learn tiene muchos modelos. Vamos a comparar varios:


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# Escalar features (necesario para algunos modelos)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_f)
X_test_scaled = scaler.transform(X_test_f)

# Definir modelos
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "Árbol de Decisión": DecisionTreeRegressor(max_depth=5, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42),
    "KNN (K=5)": KNeighborsRegressor(n_neighbors=5),
    "SVR": SVR(kernel="rbf", C=100),
}

# Entrenar y evaluar todos
resultados = []
for nombre, modelo in modelos.items():
    # Usar datos escalados para KNN y SVR
    if nombre in ["KNN (K=5)", "SVR"]:
        modelo.fit(X_train_scaled, y_train_f)
        y_pred = modelo.predict(X_test_scaled)
    else:
        modelo.fit(X_train_f, y_train_f)
        y_pred = modelo.predict(X_test_f)

    r2 = r2_score(y_test_f, y_pred)
    mae = mean_absolute_error(y_test_f, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_f, y_pred))

    resultados.append({
        "Modelo": nombre,
        "R²": r2,
        "MAE": mae,
        "RMSE": rmse,
    })

df_resultados = pd.DataFrame(resultados).sort_values("R²", ascending=False)
df_resultados.index = range(1, len(df_resultados) + 1)
print("🏆 Comparativa de modelos de regresión:")
print(df_resultados.to_string())

In [ ]:
# Visualizar comparativa
fig, ax = plt.subplots(figsize=(10, 5))

colores = ["#2E86AB" if r > 0.8 else "#F18F01" if r > 0.7 else "#C73E1D"
           for r in df_resultados["R²"]]

bars = ax.barh(df_resultados["Modelo"], df_resultados["R²"], color=colores, edgecolor="white")

for bar, r2 in zip(bars, df_resultados["R²"]):
    ax.text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
            f"{r2:.3f}", va="center", fontweight="bold", fontsize=10)

ax.set_xlabel("R² Score", fontsize=12)
ax.set_title("Comparativa de Modelos de Regresión", fontsize=14, fontweight="bold")
ax.axvline(x=0.8, color="gray", linestyle="--", alpha=0.5, label="R² = 0.80")
ax.legend()
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xlim(0, 1.05)

plt.tight_layout()
plt.show()

mejor = df_resultados.iloc[0]
print(f"\n🏆 Mejor modelo: {mejor['Modelo']} (R² = {mejor['R²']:.3f}, MAE = {mejor['MAE']:.2f} años)")

---

## 8. Clasificación: predecir categorías

Ahora vamos a cambiar de tarea: en vez de predecir un número (regresión), vamos a **predecir una categoría** (clasificación).

**Problema:** dado el PIB per cápita, la población y el año, ¿en qué continente se encuentra un país?


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Preparar datos para clasificación
df_clf = gapminder.copy()
df_clf["log_pib"] = np.log10(df_clf["gdpPercap"])
df_clf["log_pop"] = np.log10(df_clf["pop"])

# Features y target
X_clf = df_clf[["log_pib", "log_pop", "year"]].values
y_clf = df_clf["continent"].values

# Dividir
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)

print(f"📊 Clasificación: predecir continente")
print(f"  Clases: {np.unique(y_clf)}")
print(f"  Train: {len(X_train_c)}, Test: {len(X_test_c)}")
print(f"\n  Distribución de clases (test):")
for cont in np.unique(y_test_c):
    count = np.sum(y_test_c == cont)
    print(f"    {cont}: {count} ({count/len(y_test_c)*100:.1f}%)")

In [ ]:
# Entrenar Random Forest para clasificación
modelo_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
modelo_clf.fit(X_train_c, y_train_c)

# Predecir
y_pred_c = modelo_clf.predict(X_test_c)

# Accuracy
accuracy = accuracy_score(y_test_c, y_pred_c)
print(f"🎯 Accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")

# Reporte detallado
print(f"\n📊 Reporte de clasificación:")
print(classification_report(y_test_c, y_pred_c))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test_c, y_pred_c)
labels = sorted(np.unique(y_test_c))

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, cmap="Blues")

ax.set_xticks(range(len(labels)))
ax.set_yticks(range(len(labels)))
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_yticklabels(labels)
ax.set_xlabel("Predicción", fontsize=12)
ax.set_ylabel("Real", fontsize=12)
ax.set_title("Matriz de Confusión", fontsize=14, fontweight="bold")

# Agregar números
for i in range(len(labels)):
    for j in range(len(labels)):
        color = "white" if cm[i, j] > cm.max() / 2 else "black"
        ax.text(j, i, str(cm[i, j]), ha="center", va="center",
                fontsize=14, fontweight="bold", color=color)

plt.colorbar(im, ax=ax, shrink=0.8)
plt.tight_layout()
plt.show()

# Interpretación
print("📖 Cómo leer la matriz:")
print("  Filas = clase real, Columnas = clase predicha")
print("  La diagonal = predicciones correctas")
print("  Fuera de la diagonal = errores")

---

## 9. ¿Qué features importan más?

Los modelos basados en árboles nos pueden decir qué variables fueron más útiles para hacer predicciones.


In [ ]:
# Importancia de features en Random Forest (regresión)
modelo_rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
modelo_rf.fit(X_train_f, y_train_f)

importancias = pd.DataFrame({
    "feature": feature_cols,
    "importancia": modelo_rf.feature_importances_
}).sort_values("importancia", ascending=True)

fig, ax = plt.subplots(figsize=(8, 5))
ax.barh(importancias["feature"], importancias["importancia"], color="#2E86AB", edgecolor="white")
ax.set_title("Importancia de Features (Random Forest)", fontsize=14, fontweight="bold")
ax.set_xlabel("Importancia")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

for i, (_, row) in enumerate(importancias.iterrows()):
    ax.text(row["importancia"] + 0.005, i, f"{row['importancia']:.3f}",
            va="center", fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n🏆 Feature más importante: {importancias.iloc[-1]['feature']} ({importancias.iloc[-1]['importancia']:.3f})")

---

## 10. Predicciones para México 🇲🇽


In [ ]:
# Usar el mejor modelo de regresión para predecir
# Entrenar Gradient Boosting con todos los features
mejor_modelo = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
mejor_modelo.fit(X_train_f, y_train_f)

# Datos históricos de México
mexico = gapminder[gapminder["country"] == "Mexico"].copy()
mexico["log_pib"] = np.log10(mexico["gdpPercap"])
mexico["log_pop"] = np.log10(mexico["pop"])

# Crear features para México
continentes_dummies_mx = pd.get_dummies(mexico["continent"], prefix="cont", drop_first=True)
for col in [c for c in feature_cols if c.startswith("cont_")]:
    if col not in continentes_dummies_mx.columns:
        continentes_dummies_mx[col] = 0

mexico_features = pd.concat([
    mexico[["log_pib", "log_pop", "year"]].rename(columns={"year": "anio"}).reset_index(drop=True),
    continentes_dummies_mx[sorted(continentes_dummies_mx.columns)].reset_index(drop=True)
], axis=1)[feature_cols].values

# Predicciones
predicciones_mx = mejor_modelo.predict(mexico_features)

# Comparar predicción vs realidad
print(f"🇲🇽 México — Predicción vs Realidad")
print(f"{'Año':>6} {'Real':>8} {'Predicción':>12} {'Error':>8}")
print("-" * 40)
for i, (_, row) in enumerate(mexico.iterrows()):
    error = predicciones_mx[i] - row["lifeExp"]
    print(f"{row['year']:>6} {row['lifeExp']:>7.1f} {predicciones_mx[i]:>11.1f} {error:>+7.1f}")

mae_mx = np.mean(np.abs(predicciones_mx - mexico["lifeExp"].values))
print(f"\n  Error promedio para México: {mae_mx:.2f} años")

---

## 11. 🏆 Mini Proyecto: Pipeline completo de ML

Vamos a construir un pipeline profesional de principio a fin para predecir si un país tiene esperanza de vida alta o baja:


In [ ]:
# 🏆 Pipeline completo: clasificar países en "Alta" o "Baja" esperanza de vida

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# 1. Preparar datos
df_pipe = gapminder.copy()
df_pipe["log_pib"] = np.log10(df_pipe["gdpPercap"])
df_pipe["log_pop"] = np.log10(df_pipe["pop"])

# Crear target binario: esperanza de vida >= mediana global = "Alta"
mediana_global = df_pipe["lifeExp"].median()
df_pipe["categoria_ev"] = np.where(df_pipe["lifeExp"] >= mediana_global, "Alta", "Baja")

print(f"Mediana global de esperanza de vida: {mediana_global:.1f} años")
print(f"Alta (>= {mediana_global:.1f}): {(df_pipe['categoria_ev'] == 'Alta').sum()}")
print(f"Baja (< {mediana_global:.1f}): {(df_pipe['categoria_ev'] == 'Baja').sum()}")

# Features
X_pipe = df_pipe[["log_pib", "log_pop", "year"]].values
y_pipe = df_pipe["categoria_ev"].values

# 2. Dividir
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    X_pipe, y_pipe, test_size=0.2, random_state=42, stratify=y_pipe
)

# 3. Crear pipeline (escalar + modelo en un solo paso)
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("modelo", GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)),
])

# 4. Validación cruzada (más robusto que un solo train/test split)
scores_cv = cross_val_score(pipeline, X_train_p, y_train_p, cv=5, scoring="accuracy")
print(f"\n📊 Validación cruzada (5-fold):")
print(f"  Scores: {scores_cv.round(3)}")
print(f"  Media: {scores_cv.mean():.3f} ± {scores_cv.std():.3f}")

# 5. Entrenar con todos los datos de entrenamiento
pipeline.fit(X_train_p, y_train_p)
y_pred_p = pipeline.predict(X_test_p)

# 6. Evaluar
print(f"\n🎯 Accuracy en test: {accuracy_score(y_test_p, y_pred_p):.3f}")
print(f"\n{classification_report(y_test_p, y_pred_p)}")

In [ ]:
# 7. Predicciones para países específicos
paises_prueba = {
    "Mexico": {"pib": 8000, "pop": 130_000_000, "year": 2025},
    "Nigeria": {"pib": 2000, "pop": 220_000_000, "year": 2025},
    "Japón": {"pib": 40000, "pop": 125_000_000, "year": 2025},
    "Chile": {"pib": 15000, "pop": 19_500_000, "year": 2025},
}

print("🔮 Predicciones del modelo:")
print(f"  (Mediana global: {mediana_global:.1f} años)")
print()

for pais, datos in paises_prueba.items():
    features = np.array([[np.log10(datos["pib"]), np.log10(datos["pop"]), datos["year"]]])
    prediccion = pipeline.predict(features)[0]
    proba = pipeline.predict_proba(features)[0]
    confianza = max(proba) * 100

    emoji = "✅" if prediccion == "Alta" else "⚠️"
    print(f"  {emoji} {pais}: {prediccion} (confianza: {confianza:.0f}%)")
    print(f"     PIB: ${datos['pib']:,}, Población: {datos['pop']:,}")
    print()

---

## 🔥 Retos

1. **Predictor de PIB:** Invierte el problema — usa esperanza de vida, población, continente y año para predecir el PIB per cápita de un país. Compara al menos 3 modelos diferentes.

2. **Detector de continente mejorado:** Mejora el clasificador de continentes agregando más features (por ejemplo, crear features como PIB/habitante × año, o ratio población/PIB). Intenta superar el accuracy del modelo base.

3. **Sistema de recomendación de políticas:** Entrena un modelo que prediga la esperanza de vida, luego úsalo para simular: "Si México duplica su PIB per cápita, ¿cuántos años de esperanza de vida ganaría?". Haz la misma simulación para otros 5 países.


In [ ]:
# Reto 1: Predictor de PIB
# Tu código aquí 👇


In [ ]:
# Reto 2: Detector de continente mejorado
# Tu código aquí 👇


In [ ]:
# Reto 3: Sistema de recomendación de políticas
# Tu código aquí 👇


---

## 📋 Resumen

### Flujo de Machine Learning
| Paso | Código |
|------|--------|
| Dividir datos | `train_test_split(X, y, test_size=0.2)` |
| Crear modelo | `modelo = RandomForestRegressor()` |
| Entrenar | `modelo.fit(X_train, y_train)` |
| Predecir | `y_pred = modelo.predict(X_test)` |
| Evaluar | `r2_score(y_test, y_pred)` |

### Modelos principales
| Modelo | Tipo | Cuándo usarlo |
|--------|------|--------------|
| Regresión Lineal | Regresión | Relaciones lineales simples |
| Árbol de Decisión | Ambos | Datos no lineales, interpretabilidad |
| Random Forest | Ambos | Buen rendimiento general |
| Gradient Boosting | Ambos | Máximo rendimiento |
| KNN | Ambos | Datos pequeños, sin entrenamiento |

### Métricas
| Métrica | Regresión | Clasificación |
|---------|-----------|--------------|
| R² | ✅ | |
| MAE / RMSE | ✅ | |
| Accuracy | | ✅ |
| Precision / Recall | | ✅ |
| Matriz de confusión | | ✅ |

### Conceptos clave
| Concepto | Significado |
|----------|------------|
| **Overfitting** | El modelo memoriza en vez de aprender |
| **Train/Test split** | Separar datos para evaluar honestamente |
| **Cross-validation** | Evaluar con múltiples divisiones |
| **Feature engineering** | Crear mejores variables para el modelo |
| **Pipeline** | Encadenar pasos de preprocesamiento + modelo |

---

## ⏭️ ¿Qué sigue?

En el siguiente notebook exploraremos **Proyecto Final con IA** — donde aplicarás todo lo aprendido en un proyecto completo de principio a fin.

👉 [14 — Proyecto Final con IA](14_Proyecto_Final_con_IA.ipynb)

---

<p align="center">
  Hecho con ❤️ por <a href="https://culiacan.ai">Culiacan.AI</a> — Culiacán reconocida en el mundo por su talento y emprendimiento en Inteligencia Artificial
</p>
